In [3]:
import os
import numpy as np
from PIL import Image
from morphology import *
import matplotlib.pyplot as plt 

In [ ]:
input_folder="./Dataset/Classification_dataset"
output_folder="./Output/Classification_output"
compare_folder="./Compare/Classification_compare"

# input_folder="./Dataset/Grading_dataset"
# output_folder="./Output/Grading_output"
# compare_folder="./Output/Grading_compare"


if not os.path.exists(output_folder):
    os.makedirs(output_folder)

if not os.path.exists(compare_folder):
    os.makedirs(compare_folder)

for label in os.listdir(input_folder):

    label_path = os.path.join(input_folder, label)
    if not os.path.isdir(label_path):
        continue

    out_label_path = os.path.join(output_folder, label)
    if not os.path.exists(out_label_path):
        os.makedirs(out_label_path)

    compare_label_path = os.path.join(compare_folder, label)
    if not os.path.exists(compare_label_path):
        os.makedirs(compare_label_path)

    for img_name in os.listdir(label_path):
        img_path = os.path.join(label_path, img_name)
        try:
            image = Image.open(img_path).convert('RGB')
            image_np = np.array(image)
            mask = manual_color_segmentation(image_np)
            cleaned_mask = morphological_cleanup(mask)
            contour = find_mango_contour(cleaned_mask)
            if contour is None:
                print(f"警告: {img_path} 無法找到芒果輪廓，跳過此圖")
                continue
            x_min, y_min = np.min(contour, axis=0)
            x_max, y_max = np.max(contour, axis=0)
            cropped = image_np[y_min:y_max, x_min:x_max]
            cropped_img = Image.fromarray(cropped)
            save_path = os.path.join(out_label_path, img_name)
            cropped_img.save(save_path)

            # 製作比較圖
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
            ax1.imshow(image_np)
            ax1.set_title('before')
            ax1.axis('off')
            ax2.imshow(cropped)
            ax2.set_title('after')
            ax2.axis('off')
            compare_save_path = os.path.join(compare_label_path, img_name)
            plt.savefig(compare_save_path, bbox_inches='tight')
            plt.close(fig)

            print(f"已儲存切割圖片: {save_path}")
        except Exception as e:
            print(f"處理圖片失敗: {img_path}，錯誤: {e}")

已儲存切割圖片: ./Output/Classification_output\Anwar Ratool\IMG_20210630_102834.jpg
